## LSA 실습

### 1. 환경설정

In [2]:
!pip install gensim

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 73.9 MB/s eta 0:00:00


In [7]:
import numpy as np
import pandas as pd
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.datasets import fetch_20newsgroups
from gensim import corpora, matutils
from gensim.models import TfidfModel
from gensim.models import lsimodel
import matplotlib.pyplot as plt
from pprint import pprint

# NLTK 데이터 다운로드
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

### 2. 코퍼스 정의

In [4]:
# 데이터셋 로드
categories = ['alt.atheism', 'talk.religion.misc',
              'comp.graphics', 'sci.space']

newsgroups_train = fetch_20newsgroups(
    subset='train',
    categories=categories,
    remove=('headers', 'footers', 'quotes'),
    random_state=123
)

documents = newsgroups_train.data

print(f"전체 문서 수: {len(documents)}")

전체 문서 수: 2034


In [16]:
# 불용어 및 lemmatizer 설정
stopwords_en = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

# 텍스트 전처리 함수
def text_to_tokens(text):
    text = text.lower()  # 소문자 변환
    tokens = nltk.word_tokenize(text)  # 토큰화
    tokens = [lemmatizer.lemmatize(t) for t in tokens]  # 표제어 추출
    tokens = [t for t in tokens if t not in stopwords_en]  # 불용어 제거
    tokens = [t for t in tokens if re.fullmatch('[a-z]+', t)]  # 알파벳만 유지
    return tokens

# 문서 전처리
doc_tokenized = [text_to_tokens(doc) for doc in documents]
print(f"첫 번째 문서의 토큰 수: {len(doc_tokenized[0])}")
print(f"토큰 샘플: {doc_tokenized[0][:10]}")

# Gensim 사전 및 BoW 생성
dictionary = corpora.Dictionary(doc_tokenized)
bow_corpus = [dictionary.doc2bow(doc) for doc in doc_tokenized]

print(f"단어 수: {len(dictionary)}")
print(f"문서 수: {len(bow_corpus)}")

첫 번째 문서의 토큰 수: 8
토큰 샘플: ['file', 'format', 'autodesk', 'animation', 'studio', 'available', 'thanks', 'gary']
단어 수: 19886
문서 수: 2034


### 3. LSA 모델 적용 - TF-IDF 행렬
* 코퍼스를 TF-IDF 행렬을 기반으로 셍성

In [17]:
tfidf_model = TfidfModel(bow_corpus, smartirs='nfc')
tfidf_corpus = tfidf_model[bow_corpus]

In [18]:
# 세 번째 문서에서 가장 빈도가 높은 단어 찾기
doc2_bow = bow_corpus[2]
most_frequent = max(doc2_bow, key=lambda x: x[1])  # (단어ID, 빈도) 중 빈도가 최대인 것
word_id, frequency = most_frequent
word = dictionary[word_id]

print(f"세 번째 문서에서 가장 많이 등장하는 단어: '{word}' ({frequency}번)")

세 번째 문서에서 가장 많이 등장하는 단어: 'paintbrush' (3번)


In [19]:
tfidf_matrix = matutils.corpus2dense(tfidf_corpus, num_terms=len(dictionary)).transpose()
print(f"해당 단어의 인덱스: {word_id}")
print(f"해당 단어의 TF-IDF 값: {tfidf_matrix[2, word_id]:.4f}")

해당 단어의 인덱스: 71
해당 단어의 TF-IDF 값: 0.6659


* TF-IDF 행렬을 토픽 수가 2개인 LSA 모델에 적용

In [20]:
# LSA(LSI) 모델 적용
lsi_model = lsimodel.LsiModel(corpus=tfidf_corpus, id2word=dictionary, num_topics=2, random_seed=123)

### 4. 토픽별 단어 할당값

In [21]:
# 토픽별 단어 할당값 행렬
topic_word_assignment_mat = lsi_model.get_topics()
print("토픽별 단어 할당값의 행렬 표현:")
pprint(np.round(topic_word_assignment_mat, 3))

print("토픽별 중요한 단어 순서대로 표현:")
pprint(lsi_model.print_topics())

토픽별 단어 할당값의 행렬 표현:
array([[-0.019, -0.002, -0.044, ..., -0.   , -0.   , -0.   ],
       [ 0.045,  0.009,  0.082, ...,  0.   ,  0.   ,  0.   ]])
토픽별 중요한 단어 순서대로 표현:
[(0,
  '-0.153*"wa" + -0.150*"god" + -0.140*"would" + -0.120*"one" + '
  '-0.119*"people" + -0.108*"doe" + -0.107*"think" + -0.104*"space" + '
  '-0.103*"know" + -0.096*"like"'),
 (1,
  '-0.274*"god" + 0.257*"file" + 0.190*"image" + 0.161*"thanks" + '
  '0.152*"program" + 0.151*"format" + -0.140*"jesus" + 0.135*"graphic" + '
  '-0.126*"christian" + 0.112*"color"')]


In [26]:
print(np.abs(topic_word_assignment_mat[1,:]))
# 가장 큰 값의 인덱스 도출
print("가장 큰 값을 가진 인덱스:",np.argmax(np.abs(topic_word_assignment_mat[1,:])))

[0.04542264 0.00859485 0.08229202 ... 0.00028875 0.00028875 0.00028875]
가장 큰 값을 가진 인덱스: 1128


In [27]:
print("첫번째 토픽의 가장 중요한 단어:", np.argmax(np.abs(topic_word_assignment_mat[1,:])))
print("인덱스 1128의 단어는", dictionary[1128])
print("첫번째 토픽의 인덱스 1128 단어의 할당값은", topic_word_assignment_mat[0,1128])

첫번째 토픽의 가장 중요한 단어: 1128
인덱스 1128의 단어는 god
첫번째 토픽의 인덱스 1128 단어의 할당값은 -0.15018425421507628


In [29]:
# paintbrush단어의 인덱스 탐색
dictionary.token2id['paintbrush']

71

In [39]:
topic_word_assignment_mat[1,71]

np.float64(0.0024979589585771697)

In [31]:
print("두번째 토픽의 paintbrush 단어 할당값:", topic_word_assignment_mat[1,dictionary.token2id['paintbrush']])

두번째 토픽의 paintbrush 단어 할당값: 0.0024979589585771697


### 5. 문서별 토픽 할당값

In [33]:
# 문서별 토픽 할당값
doc_topic_assignment = lsi_model[bow_corpus]
print("문서별 토픽 할당값의 간단한 표현:")
pprint(list(doc_topic_assignment))

# 문서별 토픽 할당값 행렬 표현
doc_topic_assignment_mat = np.zeros(shape=(len(doc_topic_assignment), lsi_model.num_topics), dtype=float)
for doc_idx, doc_topics in enumerate(doc_topic_assignment):
    for topic_idx, topic_weight in doc_topics:
        doc_topic_assignment_mat[doc_idx, topic_idx] = topic_weight

print("문서별 토픽 할당값의 행렬 표현:")
pprint(np.round(doc_topic_assignment_mat, 3))

문서별 토픽 할당값의 간단한 표현:
[[(np.int64(0), np.float64(-0.28369167989397437)),
  (np.int64(1), np.float64(0.7299517305610559))],
 [(np.int64(0), np.float64(-1.5388211894962367)),
  (np.int64(1), np.float64(-0.07354219042247263))],
 [(np.int64(0), np.float64(-0.497495193330542)),
  (np.int64(1), np.float64(0.5448022315338422))],
 [(np.int64(0), np.float64(-0.31155417456098927)),
  (np.int64(1), np.float64(0.24358652120148303))],
 [(np.int64(0), np.float64(-1.723297501973893)),
  (np.int64(1), np.float64(-0.1299934863392989))],
 [(np.int64(0), np.float64(-1.0671799088622724)),
  (np.int64(1), np.float64(0.18622315402358436))],
 [(np.int64(0), np.float64(-0.6982444334460911)),
  (np.int64(1), np.float64(0.657912526303343))],
 [(np.int64(0), np.float64(-0.23005339158759752)),
  (np.int64(1), np.float64(-0.1313763835643121))],
 [(np.int64(0), np.float64(-0.14750426715721404)),
  (np.int64(1), np.float64(-0.021417091000079738))],
 [(np.int64(0), np.float64(-2.9656096123311126)),
  (np.int64(1), np.f

In [35]:
print("첫번째 문서의 토픽 할당값", doc_topic_assignment_mat[0,:])

첫번째 문서의 토픽 할당값 [-0.28369168  0.72995173]


* 절댓값이 두번째 토픽이 더 크므로 두번째 토픽의 영향력이 큼

### 6. 토픽별 코퍼스 내 중요도 정보

In [36]:
singular_values = lsi_model.projection.s
print("토픽별 코퍼스 내 중요도(특이값):", singular_values)

토픽별 코퍼스 내 중요도(특이값): [5.41820867 3.50596508]


In [40]:
print("첫 번째 토픽의 특이값:", singular_values[0])

첫 번째 토픽의 특이값: 5.418208665521935


* 첫번째 토픽이 두번째 토픽보다 코퍼스 내에서 더 중요(첫번째 토픽이 문서들의 주요 패턴을 더 잘 포착한다고 볼 수 있음)